In [27]:
import sagemaker
import boto3

iam_client = boto3.client('iam')
role = 'arn:aws:iam::288347194123:role/god'
sess = sagemaker.Session()

In [28]:
import transformers
import datasets
import argparse
import os

# may need to edit the model, training, test dir later

envs = dict(os.environ)
model_env = envs.get('SM_MODEL_DIR')
train_env = envs.get('SM_CHANNEL_TRAIN')
test_env = envs.get('SM_CHANNEL_TEST')

# test_dir = os.environ["SM_MODEL_DIR"]
# print(test_dir)

if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script
    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--per_device_train_batch_size", type=int, default=32)
    parser.add_argument("--model_name_or_path", type=str)

    # data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=model_env)
    parser.add_argument("--training_dir", type=str, default=train_env)
    parser.add_argument("--test_dir", type=str, default=test_env)

In [29]:
from sagemaker.huggingface import HuggingFace

# hyperparameters which are passed to the training job
hyperparameters={'epochs': 1,
                 'per_device_train_batch_size': 32,
                 'model_name_or_path': 'distilbert-base-uncased'
                 }

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge', # determines the compute power
        instance_count=1,
        role=role,
        transformers_version='4.26',
        pytorch_version='1.13',
        py_version='py39',
        hyperparameters = hyperparameters
)

In [30]:
# begin training
huggingface_estimator.fit(
  {'train': 's3://beetcode/train/',
   'test': 's3://beetcode/test/'}
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2024-05-01-17-29-30-284


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.p3.2xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.